In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import kss
from konlpy.tag import Okt  
from konlpy.tag import Kkma 
import time
from tqdm import notebook
import re

In [4]:
okt = Okt()
kkma = Kkma()

In [6]:
naver = pd.read_csv("naver_re.csv")
watcha = pd.read_csv("watcha_re.csv")

### 최종 영화를 합칠 수 있는 movie_list 만들어주기 

In [7]:
# 영화 리스트 합쳐서 중복 값 제거 
a = np.array(naver['movie_id'])
movie_list =  pd.DataFrame(a)
movie_list

,0
0,1%의 어떤 것
1,25시
2,S.W.A.T. 특수기동대
3,S.W.A.T. 특수기동대
4,S.W.A.T. 특수기동대
...,...
20956,"사랑하고 사랑받고, 차고 차이고"
20957,"사랑하고 사랑받고, 차고 차이고"
20958,"사랑하고 사랑받고, 차고 차이고"
20959,사일런스 앤드 다크니스


In [8]:
movie_list

,0
0,1%의 어떤 것
1,25시
2,S.W.A.T. 특수기동대
3,S.W.A.T. 특수기동대
4,S.W.A.T. 특수기동대
...,...
20956,"사랑하고 사랑받고, 차고 차이고"
20957,"사랑하고 사랑받고, 차고 차이고"
20958,"사랑하고 사랑받고, 차고 차이고"
20959,사일런스 앤드 다크니스


In [9]:
movie_list = movie_list[0].append(watcha['movie_id']) 
movie_list.columns = ['movie_id']

In [10]:
movie_list = pd.DataFrame(movie_list)
len(movie_list)

32298

In [11]:
movie_list= movie_list.drop_duplicates(subset=None, keep='first',ignore_index=True)

In [12]:
len(movie_list)

12038

In [13]:
movie_list

,0
0,1%의 어떤 것
1,25시
2,S.W.A.T. 특수기동대
3,갱스 오브 뉴욕
4,거짓말의 빛깔
...,...
12033,주글래 살래
12034,천방지축
12035,콜드 크릭
12036,터미널


In [14]:
movie_list = movie_list[0].append(watcha['movie_id']) 

In [15]:
movie_list = pd.DataFrame(movie_list)
movie_list.columns = ['movie_id']

In [16]:
movie_list

,movie_id
0,1%의 어떤 것
1,25시
2,S.W.A.T. 특수기동대
3,갱스 오브 뉴욕
4,거짓말의 빛깔
...,...
11332,주글래 살래
11333,천방지축
11334,콜드 크릭
11335,터미널


In [17]:
movie_list.to_csv('movie_list.csv',encoding='utf-8-sig')

In [18]:
naver = naver.fillna("")

In [19]:
watcha = watcha.fillna("")

### 네이버 중복 영화 내용 합치기 

In [20]:
#네이버 중복 데이터 합치기 
naver

,movie_id,reviews,word
0,1%의 어떤 것,살짝 억지스럽고 너무나 심하게 건전한 점에서 조금 깍였지만너무 보기 좋은 드라마고 ...,['살짝' '억지스럽다' '심하다' '건전하다' '깍이다' '보기' '좋다' '드라...
1,25시,예전리뷰 시 척박한 현실을 여과없이 드러내는 뛰어난 리얼리즘영화인동시에 진짜와...,['예전' '리뷰' '척박하다' '현실' '여과' '드러내다' '뛰어나다' '리얼리...
2,S.W.A.T. 특수기동대,개봉일에 맞춰 기다리던 를 보았는데 이젠 나아지겠지하고 인내를 하고보다가 너무 화가...,['개봉일' '맞추다' '기다리다' '보다' '나아지다' '인내' '하다' '가다'...
3,S.W.A.T. 특수기동대,수많은 액션영화에는 나름대로의 공식이 있습니다 첫번째 사격 또는 무술에 있어 타의 ...,['수많다' '액션영화' '나름' '공식' '사격' '무술' '타의' '추종' '불...
4,S.W.A.T. 특수기동대,총만 더럽게 쏴되는군 마지막에 격투씬 ㅎㅎ 말도 안되는 격투신도 나오고 해피엔딩이네...,['더럽다' '쏘다' '되다' '마지막' '격투' '안되다' '신도' '나오다' '...
...,...,...,...
20956,"사랑하고 사랑받고, 차고 차이고",애니 사랑하고 사랑받고 차고 차이고 국내 메인 예고편한글 한 메구미순정만화 같...,['애니' '사랑' '사랑받다' '차고' '차이' '국내' '메인' '예고편' '한...
20957,"사랑하고 사랑받고, 차고 차이고",사랑하고 사랑받고 차고 차이고감독쿠로야나기 토시마사출연한 메구미 시마자키 노부나가 ...,['사랑' '사랑받다' '차고' '차이' '감독' '쿠로' '나기' '토시' '마사...
20958,"사랑하고 사랑받고, 차고 차이고",사랑하고 사랑받고 차고 차이고감독쿠로야나기 토시마사출연한 메구미 시마자키 노부나가 ...,['사랑' '사랑받다' '차고' '차이' '감독' '쿠로' '나기' '토시' '마사...
20959,사일런스 앤드 다크니스,영화 사일런스 앤드 다크니스 메인 예고편 시각 장애 청각 장애가 있는 딸들시각...,['영화' '사일런스' '앤드' '다크니스' '메인' '예고편' '장애' '청각' ...


In [21]:
# sting 이 아닌 데이터를 string으로 바꿔야 join 사용 가능 
naver = naver.astype(str)

In [22]:
naver_fin = naver.groupby('movie_id',as_index=False).agg(reviews = ('reviews',','.join),
                                            word = ('word',','.join))

In [23]:
naver_fin.to_csv('naver_fin.csv',encoding='utf-8-sig',index = False)

In [24]:
naver_fin =  pd.read_csv("naver_fin.csv")

In [25]:
naver_fin

,movie_id,reviews,word
0,NaN,NaN,NaN
1,"내아들 진심으로 천국에 온것을 환영한다. """" 아..아버지",제가 아버지를 뵙습니다 너가 오기를 목마르게 기다리고 있었다 저 역시 아버지...,['아버지' '뵈다' '오기' '마르' '기다리다' '역시' '만나다' '읍니' '...
2,옆집의 친절한 이웃이 과거 전과가 있다는 걸 알게 되는 경우,그를 대하는 태도가 전과 같기 힘들 거라는 건 쉽게 예상할 수 있지요결국,['대하' '태도' '전과' '힘들다' '쉬다' '예상' '하다']
3,#아이엠히어,영화 아이엠히어 국내 메인 예고편 배두나 알랭 샤바 나여기있어요대포 대책 ...,['영화' '아이엠' '국내' '메인' '예고편' '배두나' '알랭' '샤바' '여...
4,0.0MHz,소희주인공 얘들아 내가 귀신 탐지기 사옴상엽주인공 그래 귀신 찾으러 가자친구 ...,['소희' '주인공' '귀신' '탐지기' '그렇다' '찾다' '가다' '친구' '나...
...,...,...,...
2864,헤드 오버 힐스,저두 잼있게 봤거든요 그래서 또 봤는데 처음에 느꼈던 신선함은 역시뚜욱 그냥 기분 ...,['재다' '보다' '처음' '느끼다' '신선하다' '역시' '뚜욱' '그냥' '기...
2865,형사 Duelist,두서 없이 썼던 글을 지우고 다시 씁니다물론 이 글은 프로 제 개인적인 감상문이...,['두서' '써다' '지우다' '다시' '쓰다' '프로' '개인' '감상문' '는걸...
2866,화성으로 간 사나이,제목 화성으로 간 사나이감독 김정권주연 김희선 신하균하트 점수 김정권 감독의 ...,['제목' '화성' '사나이' '감독' '김정권' '주연' '김희선' '신하균' '...
2867,황산벌,년 첫연휴를 마무리하며 서핑을 하던 중 눈에 비친 기사가 있었습니다 연휴중 방송했...,['연휴' '마무리' '하다' '서핑' '비치다' '기사' '방송' '황산벌' '중...


### Watcha 중복 데이터 합치기 

In [26]:
watcha = watcha.astype(str)

In [27]:
watcha_fin = watcha.groupby('movie_id',as_index=False).agg(reviews = ('reviews',','.join),
                                            word = ('word',','.join))

In [28]:
watcha_fin

,movie_id,reviews,word
0,#살아있다,부산행 이후만 해도 우리가 본 좀비영화가 몇 편인데 이렇게 편의적으로만 살아있다 밧...,['부산' '이후' '해도' '보다' '좀비' '영화' '편의' '살다' '밧줄' ...
1,#아이엠히어,진짜로 여기 있다는 것의 의미배두나는 프랑스어도 잘하는구나 뭐야 이 양심도 모르는 ...,['진짜' '의미' '배두나' '프랑스어' '하다' '양심' '모르다' '판타지']
2,#위왓치유,세상에서 제일 역겨운 영화드럽고 드럽고 드럽네 진짜 너무 역겹다 연유라떼 마시고...,['세상' '제일' '역겹다' '영화' '드럽다' '진짜' '연유' '라떼' '말다...
3,...ing,기억난다 년대 순정만화는 이런 느낌이었다 옛날에 사촌누나집에 놀러가면 책장 가득 채...,['기억나다' '년대' '순정만화' '이렇다' '느끼다' '옛날' '사촌누나' '놀...
4,0.0MHz,연기자면 연기를 하자 자까는 소리 하지 말고 참신함과 재미가 을 찍는 순간 ...,['연기자' '연기' '하자' '깔다' '소리' '하다' '참신하다' '재미' '찍...
...,...,...,...
11043,힐빌리의 노래,수치로 환산되지 않을 거라 믿었던 나의 배경까지 보이지 않는 자본이 되어 옴짝달싹 ...,['수치' '환산' '되다' '않다' '믿다' '배경' '보이지' '자본' '되어다...
11044,"힘내세요, 병헌씨",때론 침묵은 격려를 표현하는 가장 좋은 수단이 될때도 있다 버티는것 만들어가는것 주...,['때론' '침묵' '격려' '표현' '하다' '가장' '좋다' '수단' '되다' ...
11045,"힘을 내요, 미스터 리",다루고 있는 사건은 뭉클하지만 영화는 그 뭉클함을 그저 형식적으로 툭 건드리고 지나...,['다루다' '사건' '뭉클하다' '영화' '형식' '건드리다' '지나치다' '제발...
11046,힙스터: 안개의 덫,상당히 짜증을 불러 일으키는 영화다 성상납 안하면 너희밴드를 안쓰겠다 년대냐 연기...,['상당하다' '짜증' '불다' '일으키다' '영화' '상납' '하다' '밴드' '...


In [29]:
watcha_fin.to_csv('watcha_fin.csv',encoding='utf-8-sig',index = False)

In [30]:
watcha_fin = pd.read_csv('watcha_fin.csv')

### Merge _data 

In [31]:
movie = movie_list.merge(naver_fin, how='left', on='movie_id')

In [32]:
movie = movie.merge(watcha_fin, how='left', on='movie_id')

In [33]:
movie

,movie_id,reviews_x,word_x,reviews_y,word_y
0,1%의 어떤 것,살짝 억지스럽고 너무나 심하게 건전한 점에서 조금 깍였지만너무 보기 좋은 드라마고 ...,['살짝' '억지스럽다' '심하다' '건전하다' '깍이다' '보기' '좋다' '드라...,NaN,NaN
1,25시,예전리뷰 시 척박한 현실을 여과없이 드러내는 뛰어난 리얼리즘영화인동시에 진짜와...,['예전' '리뷰' '척박하다' '현실' '여과' '드러내다' '뛰어나다' '리얼리...,요즘 복고풍인가 해서 어릴 적에보았던 이 영화가 문득 떠올랐다 노틀담의 곱추로 유명...,['요즘' '고풍' '하다' '어리다' '보다' '영화' '문득' '떠오르다' '노...
2,S.W.A.T. 특수기동대,개봉일에 맞춰 기다리던 를 보았는데 이젠 나아지겠지하고 인내를 하고보다가 너무 화가...,['개봉일' '맞추다' '기다리다' '보다' '나아지다' '인내' '하다' '가다'...,초반에는 훈련 홍보인가 싶었었는데그래 저런 놈은 싹을 잘라야 후환이 없어그냥저냥 ...,['초반' '훈련' '홍보' '싶다' '그렇다' '저런' '자르다' '후환' '없다...
3,갱스 오브 뉴욕,너네들이 년전에 태어났어봐라 세상은 약육강식의 세상이었다 그 진리를 터득한 국가는 ...,['년전' '태어나다' '보다' '세상' '약육강식' '이다' '진리' '터득' '...,갱즈오브뉴욕에서 링컨 그림에 단검을 던져서 꽂아넣던 다니엘 데이 루이스가 여년 뒤 ...,['오브' '뉴욕' '링컨' '그림' '단검' '던지다' '꽂다' '넣다' '다니엘...
4,거짓말의 빛깔,한 바닷가 마을에서 여자아이가 성폭행 후 살해당하는 사건이 벌어지면서 범인으로 ...,['하다' '바닷가' '마을' '여자아이' '성폭행' '살해' '당하다' '사건' ...,NaN,NaN
...,...,...,...,...,...
23370,주글래 살래,NaN,NaN,심지어는 영문 제목도 로 품사 일치도 안 돼있다한마디로 이 영화는 제대로 된 게...,['영문' '제목' '품사' '일치' '돼다' '영화' '제대로' '되다' '없다'...
23371,천방지축,NaN,NaN,초호화 캐스팅에 혹해서 봤는대정말 너무 어정쩡한 장르의 표본 코미디 액션 멜로 아무...,['호화' '캐스팅' '하다' '보다' '정말' '어정쩡하다' '장르' '표본' '...
23372,콜드 크릭,NaN,NaN,점 그렇게 무섭지도 않고 그래도 나쁘진 않았다,['무섭다' '않다' '나쁘다']
23373,터미널,NaN,NaN,톰 행크스는 연기를 하는것인지 본인 그자체인지 구분못하게 만든다 톰 행크스의 매력에...,['행크스' '연기' '하다' '본인' '자체' '구분' '만들다' '매력' '무조...


In [34]:
movie['word_x']= movie['word_x'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/var/folders/gw/cgt75dcs5p5brqwyv3ptr3mm0000gn/T/ipykernel_17957/751424944.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movie['word_x']= movie['word_x'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [35]:
movie['word_y']= movie['word_y'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/var/folders/gw/cgt75dcs5p5brqwyv3ptr3mm0000gn/T/ipykernel_17957/3554630379.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movie['word_y']= movie['word_y'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [36]:
movie['word_x'] = movie['word_x'].fillna("")
movie['word_y'] = movie['word_y'].fillna("")

In [37]:
movie['reviews_x'] = movie['reviews_x'].fillna("")
movie['reviews_y'] = movie['reviews_y'].fillna("")

In [38]:
movie['word'] = movie['word_x']+ movie['word_y']
movie['reviews'] = movie['reviews_x']+ movie['reviews_y']

In [39]:
movie

,movie_id,reviews_x,word_x,reviews_y,word_y,word,reviews
0,1%의 어떤 것,살짝 억지스럽고 너무나 심하게 건전한 점에서 조금 깍였지만너무 보기 좋은 드라마고 ...,살짝 억지스럽다 심하다 건전하다 깍이다 보기 좋다 드라마 사랑스럽다 일요일 아침 하...,,,살짝 억지스럽다 심하다 건전하다 깍이다 보기 좋다 드라마 사랑스럽다 일요일 아침 하...,살짝 억지스럽고 너무나 심하게 건전한 점에서 조금 깍였지만너무 보기 좋은 드라마고 ...
1,25시,예전리뷰 시 척박한 현실을 여과없이 드러내는 뛰어난 리얼리즘영화인동시에 진짜와...,예전 리뷰 척박하다 현실 여과 드러내다 뛰어나다 리얼리즘 영화인 동시 진짜 부딪히다...,요즘 복고풍인가 해서 어릴 적에보았던 이 영화가 문득 떠올랐다 노틀담의 곱추로 유명...,요즘 고풍 하다 어리다 보다 영화 문득 떠오르다 노틀담 추로 유명하다 안소니 부리 ...,예전 리뷰 척박하다 현실 여과 드러내다 뛰어나다 리얼리즘 영화인 동시 진짜 부딪히다...,예전리뷰 시 척박한 현실을 여과없이 드러내는 뛰어난 리얼리즘영화인동시에 진짜와...
2,S.W.A.T. 특수기동대,개봉일에 맞춰 기다리던 를 보았는데 이젠 나아지겠지하고 인내를 하고보다가 너무 화가...,개봉일 맞추다 기다리다 보다 나아지다 인내 하다 가다 화가 나다 아깝다 자리 박차다...,초반에는 훈련 홍보인가 싶었었는데그래 저런 놈은 싹을 잘라야 후환이 없어그냥저냥 ...,초반 훈련 홍보 싶다 그렇다 저런 자르다 후환 없다 때우다 정도 왓챠 플레이 단순하...,개봉일 맞추다 기다리다 보다 나아지다 인내 하다 가다 화가 나다 아깝다 자리 박차다...,개봉일에 맞춰 기다리던 를 보았는데 이젠 나아지겠지하고 인내를 하고보다가 너무 화가...
3,갱스 오브 뉴욕,너네들이 년전에 태어났어봐라 세상은 약육강식의 세상이었다 그 진리를 터득한 국가는 ...,년전 태어나다 보다 세상 약육강식 이다 진리 터득 국가 지금 미국 유럽일 강대국 되...,갱즈오브뉴욕에서 링컨 그림에 단검을 던져서 꽂아넣던 다니엘 데이 루이스가 여년 뒤 ...,오브 뉴욕 링컨 그림 단검 던지다 꽂다 넣다 다니엘 데이 루이스 전기 영화 찍다 오...,년전 태어나다 보다 세상 약육강식 이다 진리 터득 국가 지금 미국 유럽일 강대국 되...,너네들이 년전에 태어났어봐라 세상은 약육강식의 세상이었다 그 진리를 터득한 국가는 ...
4,거짓말의 빛깔,한 바닷가 마을에서 여자아이가 성폭행 후 살해당하는 사건이 벌어지면서 범인으로 ...,하다 바닷가 마을 여자아이 성폭행 살해 당하다 사건 벌어지다 범인 지목 받다 화가 ...,,,하다 바닷가 마을 여자아이 성폭행 살해 당하다 사건 벌어지다 범인 지목 받다 화가 ...,한 바닷가 마을에서 여자아이가 성폭행 후 살해당하는 사건이 벌어지면서 범인으로 ...
...,...,...,...,...,...,...,...
23370,주글래 살래,,,심지어는 영문 제목도 로 품사 일치도 안 돼있다한마디로 이 영화는 제대로 된 게...,영문 제목 품사 일치 돼다 영화 제대로 되다 없다 경이 느끼다 하다 스럽다 진짜 드...,영문 제목 품사 일치 돼다 영화 제대로 되다 없다 경이 느끼다 하다 스럽다 진짜 드...,심지어는 영문 제목도 로 품사 일치도 안 돼있다한마디로 이 영화는 제대로 된 게...
23371,천방지축,,,초호화 캐스팅에 혹해서 봤는대정말 너무 어정쩡한 장르의 표본 코미디 액션 멜로 아무...,호화 캐스팅 하다 보다 정말 어정쩡하다 장르 표본 코미디 액션 멜로 건지다 개인 최...,호화 캐스팅 하다 보다 정말 어정쩡하다 장르 표본 코미디 액션 멜로 건지다 개인 최...,초호화 캐스팅에 혹해서 봤는대정말 너무 어정쩡한 장르의 표본 코미디 액션 멜로 아무...
23372,콜드 크릭,,,점 그렇게 무섭지도 않고 그래도 나쁘진 않았다,무섭다 않다 나쁘다,무섭다 않다 나쁘다,점 그렇게 무섭지도 않고 그래도 나쁘진 않았다
23373,터미널,,,톰 행크스는 연기를 하는것인지 본인 그자체인지 구분못하게 만든다 톰 행크스의 매력에...,행크스 연기 하다 본인 자체 구분 만들다 매력 무조건 기다리다 다가 아니다 어떻다 ...,행크스 연기 하다 본인 자체 구분 만들다 매력 무조건 기다리다 다가 아니다 어떻다 ...,톰 행크스는 연기를 하는것인지 본인 그자체인지 구분못하게 만든다 톰 행크스의 매력에...


In [40]:
movie = movie[['movie_id','reviews','word']]

In [41]:
movie

,movie_id,reviews,word
0,1%의 어떤 것,살짝 억지스럽고 너무나 심하게 건전한 점에서 조금 깍였지만너무 보기 좋은 드라마고 ...,살짝 억지스럽다 심하다 건전하다 깍이다 보기 좋다 드라마 사랑스럽다 일요일 아침 하...
1,25시,예전리뷰 시 척박한 현실을 여과없이 드러내는 뛰어난 리얼리즘영화인동시에 진짜와...,예전 리뷰 척박하다 현실 여과 드러내다 뛰어나다 리얼리즘 영화인 동시 진짜 부딪히다...
2,S.W.A.T. 특수기동대,개봉일에 맞춰 기다리던 를 보았는데 이젠 나아지겠지하고 인내를 하고보다가 너무 화가...,개봉일 맞추다 기다리다 보다 나아지다 인내 하다 가다 화가 나다 아깝다 자리 박차다...
3,갱스 오브 뉴욕,너네들이 년전에 태어났어봐라 세상은 약육강식의 세상이었다 그 진리를 터득한 국가는 ...,년전 태어나다 보다 세상 약육강식 이다 진리 터득 국가 지금 미국 유럽일 강대국 되...
4,거짓말의 빛깔,한 바닷가 마을에서 여자아이가 성폭행 후 살해당하는 사건이 벌어지면서 범인으로 ...,하다 바닷가 마을 여자아이 성폭행 살해 당하다 사건 벌어지다 범인 지목 받다 화가 ...
...,...,...,...
23370,주글래 살래,심지어는 영문 제목도 로 품사 일치도 안 돼있다한마디로 이 영화는 제대로 된 게...,영문 제목 품사 일치 돼다 영화 제대로 되다 없다 경이 느끼다 하다 스럽다 진짜 드...
23371,천방지축,초호화 캐스팅에 혹해서 봤는대정말 너무 어정쩡한 장르의 표본 코미디 액션 멜로 아무...,호화 캐스팅 하다 보다 정말 어정쩡하다 장르 표본 코미디 액션 멜로 건지다 개인 최...
23372,콜드 크릭,점 그렇게 무섭지도 않고 그래도 나쁘진 않았다,무섭다 않다 나쁘다
23373,터미널,톰 행크스는 연기를 하는것인지 본인 그자체인지 구분못하게 만든다 톰 행크스의 매력에...,행크스 연기 하다 본인 자체 구분 만들다 매력 무조건 기다리다 다가 아니다 어떻다 ...


In [42]:
# 최종데이터 
movie
#movie.to_csv('movie_fin.csv',encoding='utf-8-sig',index = False)

,movie_id,reviews,word
0,1%의 어떤 것,살짝 억지스럽고 너무나 심하게 건전한 점에서 조금 깍였지만너무 보기 좋은 드라마고 ...,살짝 억지스럽다 심하다 건전하다 깍이다 보기 좋다 드라마 사랑스럽다 일요일 아침 하...
1,25시,예전리뷰 시 척박한 현실을 여과없이 드러내는 뛰어난 리얼리즘영화인동시에 진짜와...,예전 리뷰 척박하다 현실 여과 드러내다 뛰어나다 리얼리즘 영화인 동시 진짜 부딪히다...
2,S.W.A.T. 특수기동대,개봉일에 맞춰 기다리던 를 보았는데 이젠 나아지겠지하고 인내를 하고보다가 너무 화가...,개봉일 맞추다 기다리다 보다 나아지다 인내 하다 가다 화가 나다 아깝다 자리 박차다...
3,갱스 오브 뉴욕,너네들이 년전에 태어났어봐라 세상은 약육강식의 세상이었다 그 진리를 터득한 국가는 ...,년전 태어나다 보다 세상 약육강식 이다 진리 터득 국가 지금 미국 유럽일 강대국 되...
4,거짓말의 빛깔,한 바닷가 마을에서 여자아이가 성폭행 후 살해당하는 사건이 벌어지면서 범인으로 ...,하다 바닷가 마을 여자아이 성폭행 살해 당하다 사건 벌어지다 범인 지목 받다 화가 ...
...,...,...,...
23370,주글래 살래,심지어는 영문 제목도 로 품사 일치도 안 돼있다한마디로 이 영화는 제대로 된 게...,영문 제목 품사 일치 돼다 영화 제대로 되다 없다 경이 느끼다 하다 스럽다 진짜 드...
23371,천방지축,초호화 캐스팅에 혹해서 봤는대정말 너무 어정쩡한 장르의 표본 코미디 액션 멜로 아무...,호화 캐스팅 하다 보다 정말 어정쩡하다 장르 표본 코미디 액션 멜로 건지다 개인 최...
23372,콜드 크릭,점 그렇게 무섭지도 않고 그래도 나쁘진 않았다,무섭다 않다 나쁘다
23373,터미널,톰 행크스는 연기를 하는것인지 본인 그자체인지 구분못하게 만든다 톰 행크스의 매력에...,행크스 연기 하다 본인 자체 구분 만들다 매력 무조건 기다리다 다가 아니다 어떻다 ...


In [43]:
len(movie)

23375

## movie_dictionary

In [44]:
movie_fin = pd.read_csv('movie_fin.csv')

In [45]:
movie_fin

,movie_id,reviews,word
0,1%의 어떤 것,살짝 억지스럽고 너무나 심하게 건전한 점에서 조금 깍였지만너무 보기 좋은 드라마고 ...,살짝 억지스럽다 심하다 건전하다 깍이다 보기 좋다 드라마 사랑스럽다 일요일 아침 하...
1,25시,예전리뷰 시 척박한 현실을 여과없이 드러내는 뛰어난 리얼리즘영화인동시에 진짜와...,예전 리뷰 척박하다 현실 여과 드러내다 뛰어나다 리얼리즘 영화인 동시 진짜 부딪히다...
2,S.W.A.T. 특수기동대,개봉일에 맞춰 기다리던 를 보았는데 이젠 나아지겠지하고 인내를 하고보다가 너무 화가...,개봉일 맞추다 기다리다 보다 나아지다 인내 하다 가다 화가 나다 아깝다 자리 박차다...
3,갱스 오브 뉴욕,너네들이 년전에 태어났어봐라 세상은 약육강식의 세상이었다 그 진리를 터득한 국가는 ...,년전 태어나다 보다 세상 약육강식 이다 진리 터득 국가 지금 미국 유럽일 강대국 되...
4,거짓말의 빛깔,한 바닷가 마을에서 여자아이가 성폭행 후 살해당하는 사건이 벌어지면서 범인으로 ...,하다 바닷가 마을 여자아이 성폭행 살해 당하다 사건 벌어지다 범인 지목 받다 화가 ...
...,...,...,...
23422,주글래 살래,심지어는 영문 제목도 로 품사 일치도 안 돼있다한마디로 이 영화는 제대로 된 게...,영문 제목 품사 일치 돼다 영화 제대로 되다 없다 경이 느끼다 하다 스럽다 진짜 드...
23423,천방지축,초호화 캐스팅에 혹해서 봤는대정말 너무 어정쩡한 장르의 표본 코미디 액션 멜로 아무...,호화 캐스팅 하다 보다 정말 어정쩡하다 장르 표본 코미디 액션 멜로 건지다 개인 최...
23424,콜드 크릭,점 그렇게 무섭지도 않고 그래도 나쁘진 않았다,무섭다 않다 나쁘다
23425,터미널,톰 행크스는 연기를 하는것인지 본인 그자체인지 구분못하게 만든다 톰 행크스의 매력에...,행크스 연기 하다 본인 자체 구분 만들다 매력 무조건 기다리다 다가 아니다 어떻다 ...


In [46]:
len(movie_fin)

23427

In [47]:
movie_dict = {}
for i in range(len(movie_fin)) :
    movie_dict[movie_fin['movie_id'][i]] = movie_fin['word'][i]

In [48]:
len(movie_dict)

12063

In [49]:
movie_dict['터미널']

'행크스 연기 하다 본인 자체 구분 만들다 매력 무조건 기다리다 다가 아니다 어떻다 자다 인생 무수 마주치다 되다 기다림 의미 보내다 중요하다 느끼다 해주다 영화 스타트랙 좋아하다 오다 여인 실제 스타트렉 비기닝 아바타 나오다 깨알 관전 포인트 약속 가치 대해 다시 생각 해보다 자극 단순하다 가장 강력하다 무기 잘알다 스티븐 스필버그 전하 잔잔하다 감동 외국인 항상 어리숙하다 뭔가 미숙 존재 보다 전형 미국인 관점 미국 갈다 떠나다 없다 마지막 조각 자세 무인도 돌아오다 공항 표류 종류 버티다 인내 견디다 삶다 나르다 갉아먹다 떨어지다 기만 누운 사람 점점 마르다 빅터 희망 심다 필요하다 확장 수확 소망 도저히 환산 막막하다 절망 줄기 찾아가다 정말 갑인 살다 순수하다 악의 완벽하다 여운 이유 진정 때문 이다'

In [50]:
movie_dict['거짓말의 빛깔']

'하다 바닷가 마을 여자아이 성폭행 살해 당하다 사건 벌어지다 범인 지목 받다 화가 자크 검블랑 아내 드리다 보네르 그녀 유혹 유명 작가 앙뜨완느 꼬네 삼각관계 심리 묘사 영화상 친밀하다 출연 여배우 미스터리 용두사미 이다 명화 재미있다 영화'

In [51]:
movie_dict['이터널 선샤인']

'그렇다 망각 자다 이다 사람 기억 안고 산다 그것 행복하다 쓰라리다 창피하다 머릿속 한구석 가지런하다 어수선하다 자리 하다 때론 불쑥 떠오르다 기도 개인 의도 의하다 성격 따르다 웃음 짓다 눈물 흘리다 머리 세차 흔들다 버리다 보존 거나 잊다 싶다 좋다 남기다 두다 평생 추억 갖다 않다 괴로움 벗어나다 어쨌든 마련 모든 가지다 없다 동물 다해 흐르다 잃다 간다 자연 선택 사라지다 수도 나쁘다 상관없다 돼다 랜덤 방식 지우다 나가다 정말 복인 터널 선샤인 등장 니체 언뜻 맞다 격언 얘기 정확하다 상황 자체 놓다 보다 실수 단순하다 잘못 이기 이전 수치심 동반 경우 많다 종류 대한 그냥 로만 반복 학습 경험 예방접종 걸리다 없어지다 소중하다 되다 마찬가지 다다 모르다 사이 실감 사실 뒤늦다 깨달 영화 인물 잃어버리다 찾다 나서다 흔적 때문 누군가 잊혀지다 싫어하다 상대 라면 더욱 만난 기억나다 묻다 인사 아니다 나라 대답 들다 섭섭하다 여고괴담 목소리 잊혀짐 공포 소재 진주 최강희 김규리 듣다 두번째 이야기 교환일기 박예진 아픔 조엘 캐리 클레멘타인 케이트 윈슬렛 힘들다 프롤로그 지난 후의 모습 화가 사랑 지워지다 헤어지다 어쩌면 당연하다 일일 지도 시술 과정 깨닫다 지겹다 싸우다 대다 그땐 행복 삭제 필사 피해 다니다 작업 최근 아프다 화나다 점점 즐겁다 변하다 강하다 남다 이쯤 자가 받다 신의 과거 일정 부분 도려내다 분명 허망 다그 사람과 가다 먹다 음식 하워드 윌킨슨 매리 커스 스트 부인 불쌍하다 마음 테두리 모양 다시 만나다 처음 몬톡 서로 비난 상대방 아주 잔인하다 일이 가장 심하다 독설 간접 떨리다 일인 시작 어떻다 알다 감정 명확하다 정의 내리다 수많다 시인 작가 가지 고민 필요 눕다 별자리 보이다 우주 개성 넘다 기발하다 차이 끄다 찰리 우프 멀다 미셸 드리다 대로 질문 던지다 주다 억지로 더없이 탁월하다 힘드다 나르다 심란하다 외국 감상 올린다 단편 리뷰 참고제목 터널 선샤인 감독 미셸 드리다 출연 캐리 케이트 윈슬렛 커스틴 던스트 마크 러팔 

### pickle ->  movie_dict

In [52]:
import pickle

In [53]:
#save
with open('movie_dict.pickle', 'wb') as f:
    pickle.dump(movie_dict, f, pickle.HIGHEST_PROTOCOL)

In [54]:
with open('movie_dict.pickle', 'rb') as f:
    movie_dict2 = pickle.load(f)

In [55]:
movie_dict2['500일의 썸머']

nan